In [4]:
import os
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

In [7]:
X_train = pd.read_csv('X_train_oversample.csv', index_col = ['Unnamed: 0'])
y_train = pd.read_csv('y_train_oversample.csv', index_col = ['Unnamed: 0'])
X_test = pd.read_csv('X_test_oversample.csv', index_col = ['Unnamed: 0'])
y_test = pd.read_csv('y_test_oversample.csv', index_col = ['Unnamed: 0'])

In [8]:
X_train['target'] = y_train
X_test['target'] = y_test

In [9]:
X_train

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target
288,1.0,-0.260822,-0.165048,0.0,0.0,0.117626,0.0,0.0,-0.789860,0.0,-0.088020,0.0,1.0,1.0,0.103881,0.040012,0
104,0.0,-0.571467,-0.165048,0.0,0.0,-0.730974,0.0,0.0,-0.172457,0.0,-0.092465,1.0,1.0,0.0,0.192557,0.040012,0
236,0.0,0.687757,-0.165048,0.0,0.0,0.146813,0.0,0.0,0.522121,1.0,0.481789,0.0,0.0,0.0,0.192557,0.020966,0
1545,0.0,0.581559,0.376523,0.0,0.0,0.505264,0.0,0.0,0.804804,1.0,-0.499022,0.0,0.0,0.0,-1.090116,1.811555,1
623,0.0,1.081611,-0.286824,0.0,1.0,-1.672989,0.0,0.0,-0.172457,0.0,1.064745,0.0,1.0,0.0,0.192557,0.040012,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,1.0,-0.260822,-0.165048,0.0,0.0,-0.388423,0.0,0.0,-0.172457,0.0,-0.660211,0.0,1.0,0.0,0.500996,0.040012,0
905,0.0,3.860780,1.165575,0.0,1.0,-1.458895,1.0,0.0,0.753647,0.0,-1.017734,0.0,0.0,0.0,-4.125589,-0.282568,1
1096,0.0,3.531296,1.165575,0.0,1.0,-1.565942,1.0,0.0,1.515606,0.0,-1.432923,0.0,0.0,0.0,-3.598476,-0.282568,1
235,0.0,-0.227538,-0.286824,0.0,0.0,0.146813,0.0,0.0,-0.095282,0.0,-0.914522,1.0,0.0,0.0,-0.115882,0.953249,0


In [10]:
cat_col_names = ['Килип1', 'Килип4', 'Тромболит3', 'ГБ6', 'АКШ', 'Килип3', 'Килип2', 'Кл.вар.ИМ1', 'Хр.оч.инф2']
num_col_names = ['Creatin', 'ФУ1', 'ИндНор1', 'Возраст', 'Pсрлао(Адср)1', 'ПИ', 'Рла1']

In [11]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig

In [12]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform = "quantile_normal"
)

trainer_config = TrainerConfig(
    auto_lr_find = False,
    batch_size = 16,
    max_epochs = 10,
    gpus = 0,
    gradient_clip_val = 10,
    load_best = True
)

model_config = NodeConfig(
    task = "classification",
    num_layers = 2,
    num_trees = 1024,
    learning_rate = 0.1,
    embed_categorical = False,
    metrics = ["accuracy"],
)

tabular_model = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = OptimizerConfig(),
    trainer_config = trainer_config,
)

In [13]:
tabular_model.fit(train = X_train, test = X_test)

C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type             | Params
-----------------------------------------------------
0 | backbone        | NODEBackbone     | 32.3 M
1 | output_response | Lambda           | 0     
2 | loss            | CrossEntropyLoss | 0     


In [47]:
pred_df = tabular_model.predict(X_test)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
1547,0.0,1.515061,-0.160335,0.0,0.0,2.153459,0.0,0.0,1.622223,1.0,0.020983,0.0,0.0,0.0,-1.094124,-0.036206,1,0.063941,0.936059,1
1171,0.0,0.629372,1.165575,1.0,1.0,-0.830675,0.0,1.0,-0.018123,0.0,0.621597,0.0,1.0,0.0,-0.733607,-0.282568,1,0.001398,0.998602,1
302,0.0,1.087158,0.562594,0.0,0.0,-1.351848,1.0,0.0,1.062349,0.0,0.080177,1.0,0.0,0.0,1.117874,-0.807730,0,0.288220,0.711780,1
108,0.0,0.415942,-0.853102,0.0,0.0,-0.495470,0.0,0.0,-1.175737,1.0,2.334086,0.0,0.0,0.0,-0.578541,0.040012,0,0.982532,0.017468,0
916,0.0,0.904099,1.165575,0.0,0.0,-1.244801,1.0,0.0,1.139524,1.0,0.732290,0.0,0.0,0.0,-0.424321,-0.282568,1,0.001002,0.998998,1


In [48]:
print_metrics(X_test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.9467787114845938 | Holdout F1: 0.9501312335958005


In [49]:
X_val = pd.read_csv('X_val.csv', index_col = 'Unnamed: 0')
y_val = pd.read_csv('y_val.csv', index_col = 'Unnamed: 0')

In [50]:
X_val['target'] = y_val

In [95]:
pred_df = tabular_model.predict(X_val)

In [97]:
print_metrics(X_val['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.9473684210526315 | Holdout F1: 0.9500000000000001


In [20]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform = "quantile_normal"
)

trainer_config = TrainerConfig(
    auto_lr_find = False,
    batch_size = 16,
    max_epochs = 10,
    gpus = 0,
    gradient_clip_val = 10,
    load_best = True
)

model_config = NodeConfig(
    task = "classification",
    num_layers = 3,
    num_trees = 1024,
    learning_rate = 0.1,
    embed_categorical = False,
    metrics = ["accuracy"],
)

tabular_model2 = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = OptimizerConfig(),
    trainer_config = trainer_config,
)

In [21]:
tabular_model2.fit(train = X_train, test = X_test)

C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type             | Params
-----------------------------------------------------
0 | backbone        | NODEBackbone     | 95.7 M
1 | output_response | Lambda           | 0     
2 | loss            | CrossEntropyLoss | 0     


In [22]:
pred_df = tabular_model2.predict(X_test)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
1547,0.0,1.515061,-0.160335,0.0,0.0,2.153459,0.0,0.0,1.622223,1.0,0.020983,0.0,0.0,0.0,-1.094124,-0.036206,1,0.134824,0.865176,1
1171,0.0,0.629372,1.165575,1.0,1.0,-0.830675,0.0,1.0,-0.018123,0.0,0.621597,0.0,1.0,0.0,-0.733607,-0.282568,1,0.000780,0.999220,1
302,0.0,1.087158,0.562594,0.0,0.0,-1.351848,1.0,0.0,1.062349,0.0,0.080177,1.0,0.0,0.0,1.117874,-0.807730,0,0.398537,0.601463,1
108,0.0,0.415942,-0.853102,0.0,0.0,-0.495470,0.0,0.0,-1.175737,1.0,2.334086,0.0,0.0,0.0,-0.578541,0.040012,0,0.979921,0.020079,0
916,0.0,0.904099,1.165575,0.0,0.0,-1.244801,1.0,0.0,1.139524,1.0,0.732290,0.0,0.0,0.0,-0.424321,-0.282568,1,0.001541,0.998459,1


In [23]:
print_metrics(X_test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.9243697478991597 | Holdout F1: 0.9268292682926829


In [83]:
pred_df = tabular_model2.predict(X_val)
print_metrics(X_val['target'], pred_df["prediction"], tag="Holdout")


Holdout Acc: 0.9487179487179487 | Holdout F1: 0.9473684210526316
